In [3]:
# %%
import torch
import torchvision.models as models
from torch.utils.data import DataLoader, ConcatDataset
import xarray as xr
import wandb

# custom
from utils.loss import MS_SSIMLoss
from utils.data import ClimatehackDataset, CustomDataset
from utils.models import create_timm_model, TimmUnet

# %%
from fastai.vision.all import *
from fastai.callback.wandb import *
from fastai.callback.tracker import *
from fastai.distributed import *
from fastai.vision.models.xresnet import *
from fastai.optimizer import ranger
from fastai.layers import Mish

# %%
import timm

In [4]:
from timm import list_models

In [7]:
list_models("*swin*")

['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window7_224_in22k',
 'swin_large_patch4_window12_384',
 'swin_large_patch4_window12_384_in22k',
 'swin_s3_base_224',
 'swin_s3_small_224',
 'swin_s3_tiny_224',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224',
 'swin_v2_cr_base_224',
 'swin_v2_cr_base_384',
 'swin_v2_cr_giant_224',
 'swin_v2_cr_giant_384',
 'swin_v2_cr_huge_224',
 'swin_v2_cr_huge_384',
 'swin_v2_cr_large_224',
 'swin_v2_cr_large_384',
 'swin_v2_cr_small_224',
 'swin_v2_cr_small_384',
 'swin_v2_cr_tiny_224',
 'swin_v2_cr_tiny_384',
 'swin_v2_cr_tiny_ns_224']

In [10]:

# %%
ARCH = "regnetv_064"
BATCH_SIZE = 32
FORECAST = 24

In [11]:
TimmUnet(
    encoder=ARCH,
    # img_size=(128, 128),
    n_in=12,
    n_out=24,
    pretrained=True,
    # bottleneck='conv',
    # self_attention=True,
)


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tpu-weights/regnetv_064_ra3-530616c2.pth" to /home/iyaja/.cache/torch/hub/checkpoints/regnetv_064_ra3-530616c2.pth


TimmUnet(
  (encoder): FeatureListNet(
    (stem): Conv2d(12, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (s1): RegStage(
      (b1): PreBottleneck(
        (norm1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (conv1): Conv2d(32, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (conv2): Conv2d(144, 144, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
        (se): SEModule(
          (fc1): Conv2d(144, 8, kernel_size=(1, 1), stride=(1, 1))
          (bn): Identity()
          (act): SiLU(inplace=True)
          (fc2): Conv2d(8, 144, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
       